In [4]:
import requests
from requests import get
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

from time import sleep
from random import randint

import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")
sys.setdefaultencoding()

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

In [118]:
rank_current = []
artist = []
title = []
label = []
incharts = []
peak = []

In [119]:
url = "https://www.offiziellecharts.de/charts/single/for-date-1512165617000"

In [120]:
requests.get(url).status_code

200

In [121]:
r = requests.get(url)

In [122]:
page = requests.get(url)
soup = bs(page.content, parser="html5lib")

In [123]:
#finding out the start and end date of the top 100 songs list
periodhelp = soup.find_all('strong')

weekdates = []
for date in periodhelp:
    weekdates.append(date.text)
    
print(f"Top 100 Songs from {weekdates[0]} - {weekdates[1]}.")


Top 100 Songs from 01.12.2017 - 07.12.2017.


In [124]:
#getting list of current rank

rankc_help = soup.find_all('span', {'class' : 'this-week'})
rank_current = [int(r.text) for r in rankc_help]

In [125]:
#getting list of artists

artist_help = soup.find_all('span', {'class' : 'info-artist'})
artist = [r.text for r in artist_help]

In [126]:
#getting list of titles

title_help = soup.find_all('span', {'class' : 'info-title'})
title = [r.text for r in title_help]

In [127]:
#getting list of labels

label_help = soup.find_all('span', {'class' : 'info-label'})
label = [r.text for r in label_help]

In [128]:
#getting list of in charts information

incharts_help = soup.find_all('span', {'class' : 'plus-data'})
incharts = [r.text.split()[2] for r in incharts_help if len(r.text.split()) > 2]

In [129]:
#getting list of overall peak position information

peak_help = soup.find_all('span', {'class' : 'plus-data'})
peak = [r.text.split()[1] for r in peak_help if len(r.text.split()) < 3]

In [130]:
dict_help = {'current_rank': rank_current,
             'artist': artist,
             'title': title,
             'label': label,
             'in_charts': incharts,
             'peak': peak,
            }

In [131]:
import datetime

df = pd.DataFrame(dict_help)
df['weekbegin'] = datetime.datetime.strptime(weekdates[0], "%d.%m.%Y")
df['weekend'] = datetime.datetime.strptime(weekdates[1], "%d.%m.%Y")
df['year'] = datetime.datetime.strptime(weekdates[0], "%d.%m.%Y").year
df['month'] = datetime.datetime.strptime(weekdates[0], "%d.%m.%Y").month
df

,current_rank,artist,title,label,in_charts,peak,weekbegin,weekend,year,month
0,1,Bausa,Was du Liebe nennst,Warner Music Germany,9,1,2017-12-01,2017-12-07,2017,12
1,2,Camila Cabello feat. Young Thug,Havana,Epic,15,2,2017-12-01,2017-12-07,2017,12
2,3,Ed Sheeran,Perfect,Atlantic,18,3,2017-12-01,2017-12-07,2017,12
3,4,Justin Bieber + BloodPop®,Friends,Republic Records,15,4,2017-12-01,2017-12-07,2017,12
4,5,Post Malone feat. 21 Savage,Rockstar,Universal,11,2,2017-12-01,2017-12-07,2017,12
...,...,...,...,...,...,...,...,...,...,...
95,96,Helene Fischer,Achterbahn,Universal Domestic Polydor,7,10,2017-12-01,2017-12-07,2017,12
96,97,Kollegah & Farid Bang,Sturmmaske auf (Intro),Banger Musik,9,1,2017-12-01,2017-12-07,2017,12
97,98,Chris Brown,Questions,RCA International,10,74,2017-12-01,2017-12-07,2017,12
98,99,Ufo361,Nice Girl 2.0,Stay High,24,16,2017-12-01,2017-12-07,2017,12


In [137]:
df.to_pickle("../pickle/charts_20171201.pkl")
df.to_csv("../csv/charts_20171201.csv", index=False)